In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_features, output_size):
        super(SimpleCNN, self).__init__()
        # 假设输入数据的形状为 (batch_size, channels, depth, sequence_length)
        # 其中 sequence_length 是变化的
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(1, 3), stride=1, padding=(0, 1))
        self.pool = nn.AdaptiveAvgPool2d((num_features, output_size))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # 展平特征向量
        x = x.view(x.size(0), -1)
        return x

# 定义模型参数
num_features = 4  # 特征数量，与输入数据的channels一致
output_size = 10  # 输出特征向量的统一长度

# 创建模型实例
model = SimpleCNN(num_features, output_size)

# 假设有两个不同长度的4维列向量
input1 = torch.randn(1, 1, 4, 25)  # 长度为15
input2 = torch.randn(1, 1, 4, 20)  # 长度为20

# 应用模型
output1 = model(input1)
output2 = model(input2)

print(output1.shape)  # 应该是 (1, 40)
print(output2.shape)  # 应该是 (1, 40)


torch.Size([1, 40])
torch.Size([1, 40])


In [14]:
import pandas as pd

# 读取Excel文件，指定没有列标题
df = pd.read_excel('data\data1.xlsx', header=None)

# 初始化一个空列表来存储结果
# 获取第六列的第一个元素作为输出数据
output_data = df.iloc[:, 5].values
results = []
i = 0
# 遍历 DataFrame，按第一列的数值进行分组
for label, group in df.groupby(0):  # 0是第一列的索引位置
    # 将第2到5列的数据转换为一维行向量
    input_data = group.iloc[:, 1:5].values.flatten()
    
    
    # 将输入数据和输出数据拼接为一个新的行向量
    row_vector = input_data.tolist() + [output_data[i]]
    
    # 将新的行向量添加到结果列表中
    results.append(row_vector)

    i=i+1

# 打印结果
print(results[-1][:-1])

Transformer 模型

In [ ]:
# 定义Transformer模型
class TransformerModel(nn.Module):
    def __init__(self, input_size, num_heads, num_layers, output_size):
        super(TransformerModel, self).__init__()
        self.input_size = input_size
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.output_size = output_size

        # 编码器层
        encoder_layers = nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # 全连接层
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, src):
        # src的形状为 (seq_length, batch_size, input_size)
        encoded = self.transformer_encoder(src)
        # 取最后一个时间步的输出
        output = self.fc(encoded[-1])
        return output